In [5]:
# COLLAPSED
import torch
from nerfstudio.cameras.cameras import Cameras, CameraType
from nerfstudio.utils import plotly_utils as vis

cx = 20
cy = 10
fx = 40
fy = 40

c2w = torch.eye(4)[None, :3, :]

camera = Cameras(fx=fx, fy=fy, cx=cx, cy=cy, camera_to_worlds=c2w, camera_type=CameraType.PANORAMA)
fig = vis.vis_camera_rays(camera)
fig.show()

print(camera.image_width, " ", camera.image_height)

tensor([40])   tensor([20])


In [2]:
import numpy as np

a = np.array([[0], [0], [0], [1]], dtype=np.float32)
b = np.array([[0,0,0,1]], dtype=np.float32)

print(a.shape)
print(b.shape)

(4, 1)
(1, 4)


In [12]:
fx, fy = 20, 20
cx, cy = 20, 10

image_height = 20
image_width = 40

image_coords = torch.meshgrid(torch.arange(image_height), torch.arange(image_width), indexing="ij")
image_coords = torch.stack(image_coords, dim=-1) + pixel_offset  # stored as (y, x) coordinates

x = image_coords[..., 1]
y = image_coords[..., 0]

coord = torch.stack([(x - cx) / fx, -(y - cy) / fy], -1)
coord_x_offset = torch.stack([(x - cx + 1) / fx, -(y - cy) / fy], -1)
coord_y_offset = torch.stack([(x - cx) / fx, -(y - cy + 1) / fy], -1)

coord_stack = torch.stack([coord, coord_x_offset, coord_y_offset], dim=0)

directions_stack_perspective = torch.stack(
                [coord_stack[..., 0], coord_stack[..., 1], -torch.ones_like(coord_stack[..., 1])], dim=-1
            )

theta = torch.sqrt(torch.sum(coord_stack**2, dim=-1))

In [13]:
image_width = 40
image_height = 20
pixel_offset = 0.5

image_coords = torch.meshgrid(torch.arange(image_height), torch.arange(image_width), indexing="ij")
image_coords = torch.stack(image_coords, dim=-1) + pixel_offset  # stored as (y, x) coordinates

u = coord_stack[..., 0] * torch.pi
v = coord_stack[..., 1] * torch.pi

sinv = torch.sin(v)
cosv = torch.cos(v)
sinu = torch.sin(u)
cosu = torch.cos(u)

xx = torch.cos(v) * torch.sin(u)
yy = torch.sin(v)
zz = torch.cos(v) * torch.cos(u)
directions_stack = torch.stack([xx, yy, zz], dim=-1)